In [ ]:
from IPython.display import clear_output
import pandas as pd
import json

import sys
sys.path.append("/src/")
from visualize import (
    show_complete_fixation_with_all_frames_all_gaze,
)

In [ ]:
folder_path = "/data/"

eval_path = folder_path + "cyprus_eval_frames.csv"
eval_path_results = folder_path + "cyprus_eval_frames_results.csv"
eval_path_results_prediction = folder_path + "cyprus_eval_frames_results_prediction.csv"

# load the names and print the categories
p = "/label_mapping.json" # path to the label mapping
with open(p, "r") as f:
    label_mapping = json.load(f)
    categories = set(label_mapping["category_mapping"].values())

In [ ]:
"""
goes throuhg all fixationas and shows the image with the fixation point
"""

CHECK = True  # set true if you already have results an want to review them

data = pd.read_csv(eval_path_results_prediction)
# Add an empty 'response' column to your DataFrame if it doesn't already exist
if "response" not in data.columns:
    data["response"] = None


for i, row in data.iterrows():
    ## get frame number
    x = row["x"]
    y = row["y"]
    session = row["session"]
    frame_nr = row["frame_nr"]
    str_frame_nr = f"{frame_nr:05d}"
    imgs_path = folder_path + session + "/video_frames_img/"

    if CHECK:
        response = row["response"]
        print(f"Frame {i+1} of {len(data)}\n{session},{str_frame_nr},{response}")
    else:
        response = ""

    # get the gaze point belonging to the fixation
    fix_df = pd.read_csv(folder_path + session + "/fixation_and_labels_leveled.csv")
    # get the fixation id
    fix_id = fix_df[fix_df["frame_nr"] == frame_nr].iloc[0]["fixation_id"]
    # show the fixation
    show_complete_fixation_with_all_frames_all_gaze(fix_df, fix_id, imgs_path)

    # if the response is in category you can eigther press enter to keep the response or change it by typing something new
    if response in categories or response in {"a", "m", "ambiguous", "merke"}:
        new_response = input("Enter your response (yes/no/maybe): ").strip().lower()
        if new_response != "":
            response = new_response
    while response not in categories and response not in {
        "a",
        "m",
        "ambiguous",
        "merke",  # list of possible responses besides the categories
    }:
        response = input("Enter your response: ").strip().lower()
        if response not in categories:
            print("Invalid response. Please enter:", categories)

    # safe response
    data.loc[i, "response"] = response

    clear_output(wait=True)

    eval_path_results_total_fix = (
        folder_path + "cyprus_eval_frames_results_total_fix.csv"
    )
    # store the results
    data.to_csv(eval_path_results_total_fix, index=False)